## Autoencoder

A type of artificial neural network used for unsupervised learning. Autoencoders can be used for anomaly detection by learning to reconstruct normal data points, and then measuring the reconstruction error for new data points. High reconstruction errors indicate anomalies.


In [18]:
import pandas as pd
import numpy as np

# Generate random revenue data for 3 clients over 45 months
num_clients = 3
num_months = 45
months = pd.date_range(start='2018-01', periods=num_months, freq='M')

# Introduce anomalies by increasing revenue by a significant factor
def introduce_anomalies(revenue_data, anomaly_factor=2):
    indices = np.random.choice(len(revenue_data), size=int(len(revenue_data) * 0.1), replace=False)
    for idx in indices:
        revenue_data[idx] *= anomaly_factor
    return revenue_data

client_data = {f"client_{i+1}": pd.DataFrame({'Month': months, 'Revenue': introduce_anomalies(np.random.uniform(800 + i * 500, 1500 + i * 500, num_months))}) for i in range(num_clients)}


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Data normalization
scaler = MinMaxScaler()
for key in client_data:
    client_data[key]['Revenue'] = scaler.fit_transform(client_data[key]['Revenue'].values.reshape(-1, 1))

# Concatenate the client data
combined_data = pd.concat([client_data[key]['Revenue'] for key in client_data])

# Train-test split
train_data, test_data = train_test_split(combined_data, test_size=0.2, shuffle=False)

# Convert to PyTorch tensors
train_data = torch.Tensor(train_data.values)
test_data = torch.Tensor(test_data.values)


In [20]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

input_dim = 1
encoding_dim = 8
model = Autoencoder(input_dim, encoding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader
train_loader = DataLoader(TensorDataset(train_data.unsqueeze(1)), batch_size=16, shuffle=True)
test_loader = DataLoader(TensorDataset(test_data.unsqueeze(1)), batch_size=16, shuffle=False)


In [21]:
# Training
epochs = 100
for epoch in range(epochs):
    for batch in train_loader:
        inputs = batch[0]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

# Testing and calculating anomaly scores
model.eval()
test_scores = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch[0]
        outputs = model(inputs)
        mse_loss = torch.mean((outputs - inputs)**2, dim=1)
        test_scores.extend(mse_loss.numpy())

# Anomaly scores
anomaly_scores = np.array(test_scores)


In [22]:
# Calculate the threshold value
threshold = np.mean(anomaly_scores) + 3 * np.std(anomaly_scores)

# Identify anomalies
anomalies = []
for i, score in enumerate(anomaly_scores):
    if score > threshold:
        anomalies.append((i, score))

print("Anomalies (Index, Anomaly Score):")
for anomaly in anomalies:
    print(anomaly)


Anomalies (Index, Anomaly Score):
(17, 0.8651904)
